In [1]:
!pip install nltk
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

### Get some sample product_ids

In [2]:
import pandas as pd
import numpy as np
import pdb
from tqdm import tqdm
import nltk
import string
import pickle

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [3]:
# Only for the first time run below code
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
metadata_filename = '/content/Movies_data(3200).csv'
metadata = pd.read_csv(metadata_filename)
metadata.sample(5)

,movieId,imdbId,tmdbId,title,original_language,overview,popularity,budget,genres,production_companies,...,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,keywords,cast,crew,poster_working_url
1673,101514,134178,45562,An Ungentlemanly Act,en,"Based on actual accounts, this film portrays t...",0.448514,0,"[{'id': 36, 'name': 'History'}, {'id': 10752, ...",[{'name': 'British Broadcasting Corporation (B...,...,1992-01-01,0,130,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",4.3,4,"[{'id': 4278, 'name': 'invasion'}, {'id': 3385...","[{'cast_id': 3, 'character': 'His Excellency, ...","[{'credit_id': '52fe46c9c3a36847f8112039', 'de...",https://image.tmdb.org/t/p/w500/qR3cDshkYnyuaf...
1929,121209,475417,15426,Unearthed,en,"In the desert area of Hat Creek, cows are foun...",6.410024,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'name': 'Cold Iron Pictures', 'id': 3916}, {...",...,2007-01-01,0,93,"[{'iso_639_1': 'en', 'name': 'English'}]",3.8,22,"[{'id': 4862, 'name': 'alien life-form'}, {'id...","[{'cast_id': 1, 'character': 'Sheriff Annie Fl...","[{'credit_id': '5381a912c3a36805a400654f', 'de...",https://image.tmdb.org/t/p/w500/AqYkR48KSFVKki...
2438,150653,2179035,212753,Empire of Dirt,en,"Like many Native families, Lena Mahikan grew u...",0.035763,0,"[{'id': 18, 'name': 'Drama'}]","[{'name': 'TÃ©lÃ©film Canada', 'id': 806}, {'n...",...,2013-11-22,0,99,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,0,"[{'id': 10322, 'name': 'native american'}, {'i...","[{'cast_id': 3, 'character': 'Lena Mahikan', '...","[{'credit_id': '590e9581c3a36864a7017399', 'de...",https://image.tmdb.org/t/p/w500/bQ5PHgOpVxRt5H...
1329,79832,28816,80364,Easy Living,en,"J.B. Ball, a rich financier, gets fed up with ...",1.170418,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","[{'name': 'Paramount Pictures', 'id': 4}]",...,1937-07-16,0,88,"[{'iso_639_1': 'en', 'name': 'English'}]",6.1,13,"[{'id': 612, 'name': 'hotel'}, {'id': 10329, '...","[{'cast_id': 1, 'character': 'Mary Smith', 'cr...","[{'credit_id': '56d74262c3a36870ac007928', 'de...",https://image.tmdb.org/t/p/w500/6m0cymeuU5BWVi...
1833,114676,1348318,37630,Protektor,en,A Czech journalist joins a Prague radio statio...,0.566863,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...","[{'name': 'Negativ s.r.o.', 'id': 6106}, {'nam...",...,2009-09-22,0,98,"[{'iso_639_1': 'cs', 'name': 'ÄŒeskÃ½'}]",5.1,5,"[{'id': 2135, 'name': 'radio'}, {'id': 2483, '...","[{'cast_id': 8, 'character': 'Hana VrbatovÃ¡',...","[{'credit_id': '52fe465d9251416c91052083', 'de...",https://image.tmdb.org/t/p/w500/uo4bXm0xqjCXQV...


In [6]:
def create_description(row):
    genres_list = [genre['name'] for genre in eval(row['genres'])]
    cast_list = [actor['name'] for actor in eval(row['cast'])]
    description = f"Title: {row['title']}, Overview: {row['overview']}, Genres: {genres_list}, Cast: {cast_list}"
    return description

create_description(metadata.iloc[3025,:])

"Title: A Billion Lives, Overview: In 20 yearsâ€™ time, there will be nearly 1.6 billion smoker around the world. Approximately 70% of smokers want to quit. The United Nationsâ€™ World Health Organisation expects a billion people will die prematurely from smoking this century. The products their doctors recommend are rarely effective and many are trapped. A new vapour technology was invented to give smokers a successful way to quit. But it was quickly demonised, and even banned in many countries. A perfect storm is brewing between smokers trying to quit, government regulators, and health charities funded by the powerful pharmaceutical industry. Director Aaron Biebert travelled across four continents interviewing doctors, scientists, and others working to save a billion lives. What he found was profound government failure, widespread corruption in the public health community and powerful subversion by big business., Genres: ['Documentary'], Cast: ['Clive Bates', 'Delon Human', 'Oliver K

In [7]:
metadata['description'] = metadata.apply(create_description, axis=1)

In [8]:
sample_desc = metadata.description.tolist()[0]
sample_desc

"Title: Jumanji, Overview: When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures., Genres: ['Adventure', 'Fantasy', 'Family'], Cast: ['Robin Williams', 'Jonathan Hyde', 'Kirsten Dunst', 'Bradley Pierce', 'Bonnie Hunt', 'Bebe Neuwirth', 'David Alan Grier', 'Patricia Clarkson', 'Adam Hann-Byrd', 'Laura Bell Bundy', 'James Handy', 'Gillian Barber', 'Brandon Obray', 'Cyrus Thiedeke', 'Gary Joseph Thorup', 'Leonard Zola', 'Lloyd Berry', 'Malcolm Stewart', 'Annabel Kershaw', 'Darryl Henriques', 'Robyn Driscoll', 'Peter Bryant', 'Sarah Gilson', 'Florica Vlad', 'June Lion', 'Brenda Lockmuller']"

In [9]:
def text_preprocessing(text: str):

    # Lowercase
    text = text.lower()
    # Remove Punctuation
    # text = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text)
    # Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    ## Stemming
    # porter = PorterStemmer()
    # stemmed = [porter.stem(word) for word in filtered_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized)

In [10]:
text_preprocessing(sample_desc)

"title : jumanji , overview : sibling judy peter discover enchanted board game open door magical world , unwittingly invite alan -- adult 's trapped inside game 26 year -- living room . alan 's hope freedom finish game , prof risky three find running giant rhinoceros , evil monkey terrifying creatures. , genre : [ 'adventure ' , 'fantasy ' , 'family ' ] , cast : [ 'robin williams ' , 'jonathan hyde ' , 'kirsten dunst ' , 'bradley pierce ' , 'bonnie hunt ' , 'bebe neuwirth ' , 'david alan grier ' , 'patricia clarkson ' , 'adam hann-byrd ' , 'laura bell bundy ' , 'james handy ' , 'gillian barber ' , 'brandon obray ' , 'cyrus thiedeke ' , 'gary joseph thorup ' , 'leonard zola ' , 'lloyd berry ' , 'malcolm stewart ' , 'annabel kershaw ' , 'darryl henriques ' , 'robyn driscoll ' , 'peter bryant ' , 'sarah gilson ' , 'florica vlad ' , 'june lion ' , 'brenda lockmuller ' ]"

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [12]:
list(metadata.description)

["Title: Jumanji, Overview: When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures., Genres: ['Adventure', 'Fantasy', 'Family'], Cast: ['Robin Williams', 'Jonathan Hyde', 'Kirsten Dunst', 'Bradley Pierce', 'Bonnie Hunt', 'Bebe Neuwirth', 'David Alan Grier', 'Patricia Clarkson', 'Adam Hann-Byrd', 'Laura Bell Bundy', 'James Handy', 'Gillian Barber', 'Brandon Obray', 'Cyrus Thiedeke', 'Gary Joseph Thorup', 'Leonard Zola', 'Lloyd Berry', 'Malcolm Stewart', 'Annabel Kershaw', 'Darryl Henriques', 'Robyn Driscoll', 'Peter Bryant', 'Sarah Gilson', 'Florica Vlad', 'June Lion', 'Brenda Lockmuller']",
 "Title: Money Train, Overview: A vengeful New York transit co

In [13]:
encoded_data = model.encode(metadata.description.tolist(), device='cuda')
encoded_data = np.asarray(encoded_data.astype('float32'))


In [14]:
encoded_title_data = model.encode(metadata.title.tolist(), device='cuda')
encoded_title_data = np.asarray(encoded_title_data.astype('float32'))


In [15]:
with open('encodings_description.pickle','wb') as file:
    pickle.dump(encoded_data, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('encodings_title.pickle','wb') as file:
    pickle.dump(encoded_title_data, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('model.pickle','wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
encoded_data.shape

(3200, 768)

In [ ]:
with open('encodings.pickle','rb') as file:
    encoding_loaded = pickle.load(file)

with open('model.pickle','rb') as file:
    model_loaded = pickle.load(file)

In [17]:
def find_similar_items(query: str, model: SentenceTransformer, encodings: np.array, top_n: int = 5):
    query_vector = model.encode([query])
    # print(query_vector)
    similarity = np.dot(encodings,query_vector.T)
    # print(similarity)
    top_items = similarity.flatten().argsort()[-top_n:][::-1]
    # print(top_items)
    # return metadata['asin'].iloc[top_items]
    return top_items

In [22]:
top_n = 10
query = "A love story between a prince and a normal girl"

top_items = find_similar_items(query=query, model=model, encodings=encoded_data, top_n=top_n)
metadata['movieId'].iloc[top_items]

2497    154400
2501    154594
2552    158426
2786    167644
2124    132315
3174    175649
603       7064
1438     86091
2325    143797
2386    147270
Name: movieId, dtype: int64

In [23]:
top_n = 10
query = "adventure genre with title jumanji"

top_items = find_similar_items(query=query, model=model, encodings=encoded_data, top_n=top_n)
metadata['movieId'].iloc[top_items]

2227    138528
0            2
1323     79238
2544    157677
572       6743
1909    119830
2509    155283
908      32705
2441    150776
673       8015
Name: movieId, dtype: int64

In [24]:
top_n = 10
query = "adventure genre with title jumanji"

top_items = find_similar_items(query=query, model=model, encodings=encoded_title_data, top_n=top_n)
metadata['movieId'].iloc[top_items]

0            2
661       7841
2742    166601
64         874
1001     46207
2818    168278
1867    116702
697       8401
2469    152680
2956    170939
Name: movieId, dtype: int64

In [25]:
num_duplicates = metadata.duplicated().sum()

print("Number of duplicated rows:", num_duplicates)

Number of duplicated rows: 0
